In [1]:
# imports
import requests
import sqlalchemy as db
import pandas as pd
from bs4 import BeautifulSoup as bs
from alchemyDB import *

In [2]:
# make DB?
engine = db.create_engine('sqlite:///bgg.sqlite')

connection = engine.connect()

metadata = db.MetaData()

Base.metadata.create_all(engine)

Test 1 request pull

In [2]:
base = 'https://boardgamegeek.com/xmlapi2/thing?id' # ex: =30549&type=boardgame,boardgameexpansion'

In [3]:
req=requests.get(base+'=1&type=boardgame,boardgameexpansion&stats=1')

In [4]:
from helpers import *

In [5]:
instantiate_games(req)

defaultdict(<function helpers.instantiate_games.<locals>.<lambda>()>,
            {'id': 1,
             'name': 'Die Macher',
             'description': 'Die Macher is a game about seven sequential political races in different regions of Germany. Players are in charge of national political parties, and must manage limited resources to help their party to victory. The winning party will have the most victory points after all the regional elections. There are four different ways of scoring victory points. First, each regional election can supply one to eighty victory points, depending on the size of the region and how well your party does in it. Second, if a party wins a regional election and has some media influence in the region, then the party will receive some media-control victory points. Third, each party has a national party membership which will grow as the game progresses and this will supply a fair number of victory points. Lastly, parties score some victory points if their p

In [97]:
soup = bs(req.content, 'xml')

In [98]:
soup.prettify()

'<?xml version="1.0" encoding="utf-8"?>\n<items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse">\n <item id="1" type="boardgame">\n  <thumbnail>\n   https://cf.geekdo-images.com/thumb/img/Y2pmp3_tPmiyJ032upHBwqD49xM=/fit-in/200x150/pic4718279.jpg\n  </thumbnail>\n  <image>\n   https://cf.geekdo-images.com/original/img/uqlrq_bQJqHpcaN7_7qocV5XfbU=/0x0/pic4718279.jpg\n  </image>\n  <name sortindex="5" type="primary" value="Die Macher"/>\n  <name sortindex="1" type="alternate" value="德国大选"/>\n  <description>\n   Die Macher is a game about seven sequential political races in different regions of Germany. Players are in charge of national political parties, and must manage limited resources to help their party to victory. The winning party will have the most victory points after all the regional elections. There are four different ways of scoring victory points. First, each regional election can supply one to eighty victory points, depending on the size of the region and how well y

In [134]:
# need to find way to loop through poll and find winning value
for vote in soup.find_all('poll')[0].find_all(attrs={'value':'Best'}):
    vote['numvotes']

'106'

In [29]:
# testing schema for adding a game from soup
game = Game(
    id = 1,
    name=soup.find('name')['value'],
    description=soup.find('description').text,
    published=int(soup.find('yearpublished')['value']),
    minplayers=int(soup.find('minplayers')['value']),
    maxplayers=int(soup.find('maxplayers')['value']),
    #suggested_numplayers,
    playingtime=int(soup.find('playingtime')['value']),
    minplaytime=int(soup.find('minplaytime')['value']),
    maxplaytime=int(soup.find('maxplaytime')['value']),
    minage=int(soup.find('minage')['value'])
    #,suggested_playerage=,
    #language_dependence=
    #,designer=""
    mechanics=[Mechanics, Mechanics, Mechanics]
)

In [33]:
# connect to DB?
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session=Session()

In [55]:
session.rollback()

In [57]:
# Not tested yet
for i in range(1,501):
    req = requests.get(base +f'={i}&type=boardgame,boardgameexpansion')
    soup = bs(req.content, 'xml')
    if soup.find('name'):
        game = Game(
            id = i,
            name=soup.find('name')['value'],
            description=soup.find('description').text,
            published=int(soup.find('yearpublished')['value']),
            minplayers=int(soup.find('minplayers')['value']),
            maxplayers=int(soup.find('maxplayers')['value']),
            #suggested_numplayers,
            playingtime=int(soup.find('playingtime')['value']),
            minplaytime=int(soup.find('minplaytime')['value']),
            maxplaytime=int(soup.find('maxplaytime')['value']),
            minage=int(soup.find('minage')['value'])
            #,suggested_playerage=,
            #language_dependence=
            #,designer=""
        )
        session.add(game)
        session.commit()
    

In [41]:
from sqlalchemy import inspect

inspector = inspect(engine)

In [42]:
print(inspector.get_table_names())

['games']


In [43]:
print(inspector.get_columns('games'))

[{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}, {'name': 'name', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'description', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'published', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'minplayers', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'maxplayers', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'suggested_numplayers', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'playingtime', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'minplaytime', 'type': I

In [58]:
df = pd.read_sql_query("SELECT * FROM games", engine)
df.tail()

,id,name,description,published,minplayers,maxplayers,suggested_numplayers,playingtime,minplaytime,maxplaytime,minage,suggested_playerage,language_dependence,designer
80,391,Ocean,The idea is for players to populate the variou...,1999,2,6,None,20,20,20,8,None,None,None
81,431,Alhambra: The Card Game,Alhambra: The Card Game is a game that both in...,1992,3,6,None,45,45,45,10,None,None,None
82,451,Beutelschneider,"Translated as Cutpurse, this is a fairly uniqu...",1997,3,4,None,30,30,30,0,None,None,None
83,470,Road to the White House,extrapolated from the preview:&#10;&#10;The es...,1992,3,6,None,240,240,240,12,None,None,None
84,490,Warangel,"Warangel is a light, fantasy wargame, comprisi...",2000,2,6,None,60,60,60,12,None,None,None


In [99]:
df

,id,name,description,published,minplayers,maxplayers,suggested_numplayers,playingtime,minplaytime,maxplaytime,minage,suggested_playerage,language_dependence,designer
0,1,Die Macher,Die Macher is a game about seven sequential po...,1986,3,5,None,240,240,240,14,None,None,None
1,2,Dragonmaster,Dragonmaster is a trick-taking card game based...,1981,3,4,None,30,30,30,12,None,None,None
2,3,Samurai,"Part of the Knizia tile-laying trilogy, Samura...",1998,2,4,None,60,30,60,10,None,None,None
3,4,Tal der Könige,When you see the triangular box and the luxuri...,1992,2,4,None,60,60,60,12,None,None,None
4,5,Acquire,"In Acquire, each player strategically invests ...",1964,2,6,None,90,90,90,12,None,None,None
5,6,Mare Mediterraneum,"In the ancient lands along the Mediterranean, ...",1989,2,6,None,240,240,240,12,None,None,None
6,7,Cathedral,"In Cathedral, each player has a set of pieces ...",1978,2,2,None,20,20,20,8,None,None,None
7,8,Lords of Creation,"In this interesting offering from Warfrog, pla...",1993,2,5,None,120,120,120,12,None,None,None
8,9,El Caballero,"Although referred to as a sequel to El Grande,...",1998,2,4,None,90,90,90,13,None,None,None
9,10,Elfenland,Elfenland is a redesign of the original White ...,1998,2,6,None,60,60,60,10,None,None,None


In [60]:
import matplotlib.pyplot as plt

In [95]:
df.query('name == "Beutelschneider"')['description'].iloc[0]

'Translated as Cutpurse, this is a fairly unique entry in the trick-taking genre.  Three of the suits are relatively normal, however, the cards that are worth points, in the form of gold coins, have the least value.  The special thing about this game is the trump suit, which could have the Assassin killing the King or the Adventurer seducing the Countess; the character interaction is very interesting aspect.&#10;&#10;'

In [94]:
print(df.query("maxplaytime == 0")['playingtime'].iloc[0])

0


In [104]:
int('5+')

ValueError: invalid literal for int() with base 10: '5+'